In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
! pip install feedparser

  Created wheel for feedparser: filename=feedparser-5.2.1-cp37-none-any.whl size=44944 sha256=3abf61598f6f6b179cffc64ad17c04afac16263a50b87ca7be5e95af99918d87
  Stored in directory: C:\Users\trell\AppData\Local\pip\Cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser


In [35]:
print(df["summary"].values[0])

After a fiery overnight stand-off, some protesters try to leave but police react with tear gas.


In [45]:
posts = []
for url in rawrss:
    feed = feedparser.parse("https://timesofindia.indiatimes.com/rssfeedstopstories.cms")
    for post in feed.entries:
        posts.append((post.title, post.link, post.summary))

In [46]:
df = pd.DataFrame(posts, columns=['title', 'link', 'summary'])

In [47]:
df.head()

,title,link,summary
0,People's hearts can be won without rushing to ...,https://timesofindia.indiatimes.com/india/dist...,With disruptions in Rajya Sabha in the past hi...
1,"Pawar: Sena, BJP have to choose their paths",https://timesofindia.indiatimes.com/india/shar...,"Speaking to media in Parliament, Nationalist C..."
2,JNU protest live: 3 metro stations in Delhi shut,https://timesofindia.indiatimes.com/city/delhi...,Delhi Police on Monday stopped hundreds of JNU...
3,Live: Sharad Pawar meets Sonia at her residence,https://timesofindia.indiatimes.com/india/maha...,More than 20 days after declaration of electio...
4,People's hearts can be won without rushing to ...,https://timesofindia.indiatimes.com/india/dist...,With disruptions in Rajya Sabha in the past hi...


In [44]:
print(df["summary"].values[0])

With disruptions in Rajya Sabha in the past hindering passage of legislations, PM Modi on Monday said the Upper House was essential for checks and balance in democracy but a distinction needs to be drawn between checking and clogging. Speaking on the occasion of 250th session of Rajya Sabha, he lauded the NCP and the BJD for never rushing to the well of the House.


In [49]:
# split the the text in the articles into sentences
sentences = []
for s in df['summary']:
  sentences.append(sent_tokenize(s)) 

In [50]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [51]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [54]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [55]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [61]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [63]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [64]:
len(sentence_vectors)

44

In [65]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [68]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [69]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [76]:
# Specify number of sentences to form the summary
sn =22

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

More than 20 days after declaration of election results, there is still no government in Maharashtra.
More than 20 days after declaration of election results, there is still no government in Maharashtra.
More than 20 days after declaration of election results, there is still no government in Maharashtra.
More than 20 days after declaration of election results, there is still no government in Maharashtra.
He said he will meet Sonia Gandhi later in the day at her residence.
He said he will meet Sonia Gandhi later in the day at her residence.
He said he will meet Sonia Gandhi later in the day at her residence.
He said he will meet Sonia Gandhi later in the day at her residence.
With disruptions in Rajya Sabha in the past hindering passage of legislations, PM Modi on Monday said the Upper House was essential for checks and balance in democracy but a distinction needs to be drawn between checking and clogging.
With disruptions in Rajya Sabha in the past hindering passage of legislations, PM